Ну что же, приступим. Подключаем все необходимые библиотеки.

In [ ]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from pymystem3 import Mystem

Загружаем стоп-слова для русского языка. Для лемматизации я использовал pymystem3, и хочу сказать, что этот лемматизатор мне очень понравился - он довольно четко и правильно приводил слово к начальной форме. С Наташей у меня возникли проблемы, а pymorphy2, насколько я понял, уже не поддерживается, и с ним само собой возникли проблемы.

In [ ]:
nltk.download('stopwords')
stopwords_r = set(stopwords.words("russian"))

mystem = Mystem()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


Это, собственно, моя функция лемматизации.

In [ ]:
def lemmatize_russian(text):
  lemmatized_text = mystem.lemmatize(text)
  return ''.join([token.strip() for token in lemmatized_text])

Дальше идёт моя функция для предобработки данных. Как и объясняли на лекции, я привожу всё к одному регистру, решаю оставить только цифры и буквы русского и английского алфавита, поскольку в других символах ценной информации я не заметил, хотя, возможно, она и была. Другие языки я тоже не нашёл) На токены разбиваю с помощью функции casual_tokenize, мне понравилось, как она это делает. Забегая вперёд скажу, что я пробовал различать русский и английский языки и для первого применять лемматизацию, а для второго - стемминг. Но скор увелчился, когда ко всему применял только лемматизацию. Как-то так.

In [ ]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^а-яa-z0-9 ]', ' ', text)
    tokens = casual_tokenize(text)
    tokens = [token for token in tokens if len(token) > 1 and token not in stopwords_r]

    processed_tokens = []
    for token in tokens:
      processed_tokens.append(lemmatize_russian(token))

    return ' '.join(processed_tokens)

Дальше я загружаю оба датасета. Одно значение в тренировочных данных было пропущено, решил заменить просто пустой строкой. Также, я решил объединить в одну фичу url и title. Ещё добавил 2 новые фичи, как длина url и длина title.

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['url'] = train['url'].fillna('')
train['title'] = train['title'].fillna('')

train['url_length'] = train['url'].apply(len)
train['title_length'] = train['title'].apply(len)

test['url'] = test['url'].fillna('')
test['title'] = test['title'].fillna('')

test['url_length'] = test['url'].apply(len)
test['title_length'] = test['title'].apply(len)

train['text'] = train['url'] + ' ' + train['title']
test['text'] = test['url'] + ' ' + test['title']


# Препроцессинг в деле
train['text_clean'] = train['text'].apply(preprocess_text)
test['text_clean'] = test['text'].apply(preprocess_text)

Решил вывести и посмотреть, как там дело обстоит с меткой 1)) К такому я был готов.

In [ ]:
train_with_label_1 = train[train['label'] == 1]
print(train_with_label_1[['text_clean', 'label']].tail(50))

                                               text_clean  label
134916  fuckup xxx pornstars fucking dorm guys watch h...      1
134927  xuk life эротика голый susie фото 14 xuk ru уб...      1
134935  pornboxo com hotkinkyjo horse dildo free mastu...      1
134943  seksart com секскомикс дождаться свадьба порно...      1
134946  russkoe 365 xxx молодой телка трудный дело люб...      1
134949  www pornolenta co порно няня онлайн бесплатно ...      1
134950  www nimila net sunny leone in red lace lingeri...      1
134957  okporno tv нора дэвис красивый брюнетка нижний...      1
134971  ruero net скачать порно видео очаровательный к...      1
134974  pornpixfree com wet pussy porn pics for mobile...      1
134977                pornovidonline com анал порно видео      1
134980  th vlip lv bigo live beautiful supper thai gir...      1
134985  newyoungfogies com xnxx videos pornф xnxx de q...      1
135003  biqle ru ideal body girl gracie lewis high end...      1
135021     erkiss live де

И, пожалуй, самое интересное. В самой первой реализации я использовал TF-IDF и получил высокий скор. У меня даже не было мысли переходить на "мешок слов". Но с первым скор как-то очень медленно рос, чтобы я ни делал, поэтому решли попробовать что-то новое, и этим стал CountVectorizer. И, как не сложно догадаться, скор вырос))) Интересно, почему именно так.

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,5), max_features=10000)
X_train = vectorizer.fit_transform(train['text_clean'])
X_test = vectorizer.transform(test['text_clean'])

# Объединяем все признаки
from scipy.sparse import hstack
X_train_final = hstack([X_train, train[['url_length', 'title_length']].values])
X_test_final = hstack([X_test, test[['url_length', 'title_length']].values])

Здесь ничего интересного - обычная логистическая регрессия. Другие алгоритмы обучения я даже не рассматривал, потому что хотел получить высокий скор, используя только её.

In [ ]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train_final, train['label'])

LogisticRegression(max_iter=10000)

И последнее - это постобработка. Тут мне пришлось вспомнить много "хороших" слов, которые я знаю)) По сути, недостающая точность уже добиралась здесь. Не знаю, хорошо это или плохо, но как факт. Но я думаю, что это всё равно полезный этап в итоговой разметке.

In [ ]:
# Список запрещённых слов
forbidden_words = ['porn', 'порн', 'секс', 'adult', 'эротик', 'порнографический', 'ебля',
                   'ёбля', 'porno', 'минет', 'пизд', 'пёзд', 'ебать', 'лесбийский',
                   'хуй', 'кремпай', 'кремпа', 'кримпа', 'кримпай', 'порно', 'пизда', 'sex',
                   'сиська', 'жопа', 'blowjob', 'tits', 'milf', 'milfa', 'милфа', 'трахать', 'boob',
                   'куни', 'кунилингус', 'fuck', 'fucking']

# Дополнительные подстроки для поиска
additional_forbidden_substrings = ['xxx', 'sex', 'porn']

def postprocess_predictions(texts, preds):
    for i, text in enumerate(texts):
        if any(forbidden_word in text.lower() for forbidden_word in forbidden_words):
            preds[i] = 1
        elif any(substring in token.lower() for token in text.split() for substring in additional_forbidden_substrings):
            preds[i] = 1
    return preds

In [ ]:
preds = model.predict(X_test_final)
preds = postprocess_predictions(test['text_clean'], preds)

In [ ]:
submission = pd.DataFrame({
    'ID': test['ID'],
    'label': preds
})
submission.to_csv('submission.csv', index=False)

Как итог, хотелось бы сказать, чего получилось достичь, а чего не совсем. Главной целью было преодолеть скор в 0.97, и с этой задачей я успешно справился. Не получилось же забраться в 0.98)) Я думаю, что можно было бы использовать ещё такие признаки, как домен, потому что я заметил, что у сайтов 18+ зачастую домен 1-го уровня это .org, .net или .com. И вообще у них обычно странные названия, за которые точно можно было зацепиться. Наверное, ещё можно было бы поиграться с разными настройками TF-IDF или мешка слов.